In [4]:
# import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
udp_2017 = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/reference/udp_2017results.csv')
# display(udp_2017.head())
# rec1_vars = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/all_vars_rec1.csv')
# display(rec1_vars.head())
rec1_vars_dummies = pd.read_csv('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/regression/demog_with_cra_dummies_201314.csv')
display(rec1_vars_dummies.head())
rec1_vars_dummies.columns

,Geoid,Percent of population 25 years and over with Bachelor's degree,% below poverty level,% Single Family,% Owner Occupied,Percent NH White alone,Percent NH Black or African African alone,Percent NH Asian alone,Percent Hispanic,Total number of housing units,...,CRA,County_Alameda County,County_Contra Costa County,County_Marin County,County_Napa County,County_San Francisco County,County_San Mateo County,County_Santa Clara County,County_Solano County,County_Sonoma County
0,6001400100,0.345,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,...,0,1,0,0,0,0,0,0,0,0
1,6001400300,0.321,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,...,0,1,0,0,0,0,0,0,0,0
2,6001400400,0.440,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,...,0,1,0,0,0,0,0,0,0,0
3,6001400500,0.281,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,...,1,1,0,0,0,0,0,0,0,0
4,6001400600,0.519,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,...,1,1,0,0,0,0,0,0,0,0


Index(['Geoid',
       'Percent of population 25 years and over with Bachelor's degree',
       '% below poverty level', '% Single Family', '% Owner Occupied',
       'Percent NH White alone', 'Percent NH Black or African African alone',
       'Percent NH Asian alone', 'Percent Hispanic',
       'Total number of housing units', 'Median home value', 'Median income',
       'Total Loans', 'Year_x', 'Tract_y', 'type', 'CRA Eligible', 'CRA',
       'County_Alameda County', 'County_Contra Costa County',
       'County_Marin County', 'County_Napa County',
       'County_San Francisco County', 'County_San Mateo County',
       'County_Santa Clara County', 'County_Solano County',
       'County_Sonoma County'],
      dtype='object')

#### Get Loan Data

In [6]:
parsed_data_path = '/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data_1/'
files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2015", "2016", "2017"]]
dfs = []

for file in files:
    df_lol = []
    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]

        df_lol.append(row_list)
        
                
all_20151617 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2015-2017", "% Loans to MHI borrowers"])
all_20151617.head(30)
    


files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2013", "2014"]]
dfs = []
for file in files:
    df_lol = []

    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]
        df_lol.append(row_list)
                
all_201314 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2013-2014", "% Loans to MHI borrowers"])

all_201314.head()


    
files = [f for f in os.listdir(parsed_data_path) if f[-3:] == "csv" and f.split('_')[1] in ["2009"]]
df_lol = []
dfs = []
for file in files:
    df_lol = []
    df = pd.read_csv(parsed_data_path+file)[["Geoid","Year", "type", "# HI borrower, LI tract", "# HI borrower, HI tract", 
                 "# LI borrower, LI tract", "# LI borrower, HI tract"]]
    dfs.append(df)
    all_years = pd.concat(dfs, axis=0)
    for tract, tract_df in all_years.groupby("Geoid"):
        total_loans = (tract_df["# HI borrower, LI tract"].sum()+ tract_df["# HI borrower, HI tract"].sum()
        +tract_df["# LI borrower, LI tract"].sum()+ tract_df["# LI borrower, HI tract"].sum())
        pct_HIB = (tract_df["# HI borrower, LI tract"].sum()+tract_df["# HI borrower, HI tract"].sum())/total_loans
        row_list = [tract, total_loans ,pct_HIB]
        df_lol.append(row_list)
                
all_2009 = pd.DataFrame(df_lol, columns=["Geoid",  "Total loans 2009", "% Loans to MHI borrowers"])
all_2009.head()


print(len(all_2009))


1378


In [9]:
#these are duplicates
len(all_201314[all_201314["Geoid"].duplicated(keep='first')])
#1540 duplicates. if I get rid of these the numbers will match better
len(all_20151617[all_20151617["Geoid"].duplicated(keep='first')])
all_2009.head()




,Geoid,Total loans 2009,% Loans to MHI borrowers
0,6001400100,55,0.981818
1,6001400200,20,1.000000
2,6001400300,64,0.906250
3,6001400400,48,0.979167
4,6001400500,57,0.877193


#### Merging Data for 2013-2014 model

In [10]:
rec1_udp_merge = pd.merge(rec1_vars_dummies, udp_2017, how='left', right_on='geo_fips', left_on='Geoid')
rec1_udp_loans_merge = pd.merge(rec1_udp_merge, all_201314, how='left', on='Geoid')
display(rec1_udp_merge.head())
print("rec1vars: ", len(rec1_vars_dummies))
print("udp: ", len(udp_2017))
print("loans tract: ", len(all_201314)) #this has all the tracts not just the ones with neighbors. also can't use this
#as is because it has one row for each year, and I need to compute new proportions using the totals for the periods
print("merged 1: ", len(rec1_udp_merge))
print("merged 2: ", len(rec1_udp_loans_merge))
# rec1_udp_loans_merge.head()

,Geoid,Percent of population 25 years and over with Bachelor's degree,% below poverty level,% Single Family,% Owner Occupied,Percent NH White alone,Percent NH Black or African African alone,Percent NH Asian alone,Percent Hispanic,Total number of housing units,...,County_Marin County,County_Napa County,County_San Francisco County,County_San Mateo County,County_Santa Clara County,County_Solano County,County_Sonoma County,geo_fips,Typology,old disp type
0,6001400100,0.345,3.1,0.912439,0.895260,0.769150,0.041435,0.110724,0.007312,1439,...,0,0,0,0,0,0,0,6001400100,MHI - Advanced Exclusion,8.0
1,6001400300,0.321,6.9,0.478758,0.405179,0.714142,0.099114,0.075544,0.085214,2801,...,0,0,0,0,0,0,0,6001400300,LI - Not Losing Low Income Households,2.0
2,6001400400,0.440,4.0,0.555446,0.433809,0.698082,0.092588,0.078143,0.062988,2020,...,0,0,0,0,0,0,0,6001400400,MHI - Advanced Gentrification,6.0
3,6001400500,0.281,6.0,0.444380,0.437722,0.419526,0.346635,0.066241,0.115988,1735,...,0,0,0,0,0,0,0,6001400500,LI - Ongoing Gentrification and/or Displacement,4.0
4,6001400600,0.519,26.4,0.632653,0.470756,0.338305,0.473150,0.032220,0.059666,784,...,0,0,0,0,0,0,0,6001400600,LI - Ongoing Gentrification and/or Displacement,4.0


rec1vars:  842
udp:  2131
loans tract:  1549
merged 1:  842
merged 2:  842


In [11]:
# rec1_udp_loans_merge.head(20)
# rec1_udp_merge.head(20)
# all_201314.head(20)

#### Independent Variables

In [12]:
def encode_udp(row):
    if (row == 'LI - Ongoing Gentrification and/or Displacement'):
        return 6
    if (row == 'LI - At Risk of Gentrification and/or Displacement'):
        return 4    
    elif ('MHI - At Risk of Exclusion'):
        return 3
    elif ('LI - Not Losing Low Income Households'):
        return 2
    else:
        return 0


In [52]:
rec1_udp_loans_merge["UDP code"] = rec1_udp_loans_merge["Typology"].apply(lambda x: encode_udp(x))



#### OLS Regression

#### 2013_2014 Models

In [56]:
x_1314 =  rec1_udp_loans_merge[["CRA", "Total Loans", 'County_San Mateo County',
    'County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County', "Above average Loans to MHI borrowers"]]
y_1314 =  rec1_udp_loans_merge["UDP code"]

x_1314_constant = sm.add_constant(x_1314)
x_1314_constant
r = sm.regression.linear_model.OLS(y_1314, x_1314_constant)
results_r = r.fit()
print(results_r.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/new_regression_models/201314_udp.html', 'w') as outfile:
    outfile.write(results_r.summary().as_html())

                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     18.40
Date:                Sun, 10 Mar 2019   Prob (F-statistic):           3.79e-33
Time:                        17:46:43   Log-Likelihood:                -1267.9
No. Observations:                 842   AIC:                             2560.
Df Residuals:                     830   BIC:                             2617.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [61]:
all_x_1314 =  rec1_udp_loans_merge[["CRA", "% Loans to MHI borrowers", "% Single Family",
    "% below poverty level", "Percent NH White alone", "Total number of housing units",
     "Percent of population 25 years and over with Bachelor's degree", "Median home value", "Median income",
            'County_San Mateo County','County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County']]
# "Total Loans",
all_y_1314 =  rec1_udp_loans_merge["UDP code"]

      
  

In [62]:
all_x_1314_constant = sm.add_constant(all_x_1314)
m = sm.regression.linear_model.OLS(all_y_1314, all_x_1314_constant)
results_m = m.fit()
print(results_m.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/new_regression_models/201314_udp_all.html', 'w') as outfile:
    outfile.write(results_m.summary().as_html())
    
    

                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     18.32
Date:                Sun, 10 Mar 2019   Prob (F-statistic):           8.54e-47
Time:                        17:53:10   Log-Likelihood:                -1224.8
No. Observations:                 842   AIC:                             2486.
Df Residuals:                     824   BIC:                             2571.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [63]:
vif = pd.DataFrame()
no_counties = all_x_1314.loc[:, ~all_x_1314.columns.isin(["County_Alameda County", "County_Contra Costa County", "County_Marin County", 
          "County_San Francisco County", "County_San Mateo County", "County_Santa Clara County",
              "County_Solano County", "County_Sonoma County", "% Owner Occupied"])]
vif["VIF Factor"] = [variance_inflation_factor(no_counties.values, i) for i in range(no_counties.shape[1])]
vif["features"] = no_counties.columns
vif.round(2)


vif.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/collinearity/VIF_201314_udp.csv")


#### 2015-2017 Models

In [58]:
x_151617 =  rec1_udp_loans_merge[["CRA", "% Loans to MHI borrowers", "Total Loans", 'County_San Mateo County',
    'County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County']]
y_151617 =  rec1_udp_loans_merge["UDP code"]

x_151617_constant = sm.add_constant(x_151617)
r = sm.regression.linear_model.OLS(y_151617, x_151617_constant)
results_r = r.fit()
print(results_r.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/new_regression_models/20151617_udp.html', 'w') as outfile:
    outfile.write(results_r.summary().as_html())

                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.186
Method:                 Least Squares   F-statistic:                     18.50
Date:                Sun, 10 Mar 2019   Prob (F-statistic):           2.55e-33
Time:                        17:47:44   Log-Likelihood:                -1267.5
No. Observations:                 842   AIC:                             2559.
Df Residuals:                     830   BIC:                             2616.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [64]:
all_x_151617 =  rec1_udp_loans_merge[["CRA", "% Loans to MHI borrowers", "% Single Family",
     "% below poverty level", "Percent NH White alone", "Total number of housing units",
     "Percent of population 25 years and over with Bachelor's degree", "Median home value", "Median income",
            'County_San Mateo County','County_Santa Clara County','County_Solano County','County_Sonoma County','County_Alameda County', 
            'County_Contra Costa County','County_Marin County', 'County_San Francisco County']]
all_y_151617 =  rec1_udp_loans_merge["UDP code"]
all_x_151617_constant = sm.add_constant(all_x_151617)
r = sm.regression.linear_model.OLS(all_y_151617, all_x_151617_constant)
results_r = r.fit()
print(results_r.summary())
with open('/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/new_regression_models/20151617_udp_all.html', 'w') as outfile:
    outfile.write(results_r.summary().as_html())


                            OLS Regression Results                            
Dep. Variable:               UDP code   R-squared:                       0.274
Model:                            OLS   Adj. R-squared:                  0.259
Method:                 Least Squares   F-statistic:                     18.32
Date:                Sun, 10 Mar 2019   Prob (F-statistic):           8.54e-47
Time:                        17:53:25   Log-Likelihood:                -1224.8
No. Observations:                 842   AIC:                             2486.
Df Residuals:                     824   BIC:                             2571.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [65]:
vif = pd.DataFrame()
no_counties = all_x_151617.loc[:, ~all_x_151617.columns.isin(["County_Alameda County", "County_Contra Costa County", "County_Marin County", 
          "County_San Francisco County", "County_San Mateo County", "County_Santa Clara County",
              "County_Solano County", "County_Sonoma County", "% Owner Occupied"])]
vif["VIF Factor"] = [variance_inflation_factor(no_counties.values, i) for i in range(no_counties.shape[1])]
vif["features"] = no_counties.columns
vif.round(2)
vif.to_csv("/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/results/collinearity/VIF_20151617_udp.csv")
vif



,VIF Factor,features
0,2.784431,CRA
1,45.805729,% Loans to MHI borrowers
2,8.536656,% Single Family
3,3.341812,% below poverty level
4,7.112089,Percent NH White alone
5,8.335243,Total number of housing units
6,18.512644,Percent of population 25 years and over with B...
7,26.991477,Median home value
8,22.517372,Median income
